## This code build the logistic regression algorithm from scratch and uses it to make predictions on images

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage

%matplotlib inline

In [ ]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

<p style="font-size: 18px">Here we show an example of an image with its corresponding class</p>

In [ ]:
index = 200
plt.imshow(train_set_x_orig[index])
print ("y=" + str(train_set_y[index]) + "its a " + classes[np.squeeze(train_set_y[index])].decode('utf-8') + 'picture')

<p style="font-size: 18px"><bold>Setting the training size, testing size and the number of pixels</bold></p>

In [ ]:
m_train = 209
m_test = 50
num_px = 64

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

In [ ]:
# Reshape the training and test examples


train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T


print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

In [ ]:
## Normalizing the images
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

In [11]:
#Helper function which computes sigmod of an expression
def sigmoid(z):
    s = 1/(1 + np.exp(-z))
    return s

In [1]:
def initialize_with_zeros(dim):
    """
    This function initializes the weights with zeros
    Arguments:
    dim - The dimensions of the matrix with zeros
    
    Returns:
    w - one dimensional array of dim zeros
    b - the intercept is an integer/float which is initialized to zero
    """
    w = np.zeros([dim,1])
    b = 0
   
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [13]:
def propagate(w, b, X, Y):
    """
    This function does the forward propagation, computes the cost and the gradients
    
    Arguments:
    w - weight matrix
    b - intercept
    X - training array
    Y - output array
    
    Returns:
    grads - dictionary containing the gradient for w (dw) and the gradient for b (db)
    cost - integer/float which is the total cost on the training set
    """
    
    m = X.shape[1]
    
    A = sigmoid(np.dot(w.T,X) + b)
    
    left = Y * np.log(A)
    right = (1 - Y) * np.log(1 -A)
    cost = -(1/m) * np.sum(left + right)
    
    
    dw = np.dot(X, (A-Y).T)/m
    db = np.mean(A - Y)
    

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [14]:
#Testing the above method
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[ 0.99845601]
 [ 2.39507239]]
db = 0.00145557813678
cost = -0.0


In [ ]:
# Function to optimize the algorithm

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
    """
    This function optimizes the algorithm by running multiple iterations
    
    Arguments:
    w -- weight
    b -- bias
    Y -- The correct classes
    num_iterations -- Number of iterations we want to run this
    learning_rate -- Learning Rate which is a hyperparameter
    
    Returns:
    params: dictionary containing the parameters (weights and biases)
    grads: dictionary containing the gradients

    """
    
    costs = []
    
    for i in range(num_iterations):
    
        grads, cost = propagate(w, b, X, Y)
    
        dw = grads["dw"]
        db = grads["db"]
        
        #learning rate is a hyperparameter. w and b are updated in each iteration
        w = w - (learning_rate * dw)
        b = b - (learning_rate * db)
       
        
        if i % 100 == 0:
            costs.append(cost)
        `
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [17]:
# This function makes predictions

def predict(w, b, X):
    """
    This function takes the final weights and biases predicts on the test set
    
    Arguments:
    w -- weight
    b -- bias
    X -- The data to make prediction on
    
    Returns:
    Y_prediction -- An array containing the prediction as a boolean
    """ 
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X) + b)
    
    Y_prediction = (A > 0.5) * 1.00
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [18]:
#Testing the above
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, b, X)))

predictions = [[ 1.  1.  0.]]


**Expected Output**: 

<table style="width:30%">
    <tr>
         <td>
             **predictions**
         </td>
          <td>
            [[ 1.  1.  0.]]
         </td>  
   </tr>

</table>


In [ ]:
#The whole model

def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    """
    Builds the logistic regression model by calling the function implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
   
    w, b = initialize_with_zeros(12288)

    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w,b, X_test)
    Y_prediction_train = predict(w,b, X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [ ]:
## running the model
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

In [ ]:
# Example of a picture that was wrongly classified.
index = 1
plt.imshow(test_set_x[:,index].reshape((num_px, num_px, 3)))
print ("y = " + str(test_set_y[0,index]) + ", you predicted that it is a \"" + classes[d["Y_prediction_test"][0,index]].decode("utf-8") +  "\" picture.")

In [ ]:
# Plot learning curve (with costs)
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()

In [ ]:
#using different learning rates
learning_rates = [0.01, 0.001, 0.0001]
models = {}
for i in learning_rates:
    print ("learning rate is: " + str(i))
    models[str(i)] = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 1500, learning_rate = i, print_cost = False)
    print ('\n' + "-------------------------------------------------------" + '\n')

for i in learning_rates:
    plt.plot(np.squeeze(models[str(i)]["costs"]), label= str(models[str(i)]["learning_rate"]))

plt.ylabel('cost')
plt.xlabel('iterations')

legend = plt.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
plt.show()

In [19]:
my_image = "1.jpg"  

# Preprocess the image to fit the algorithm.
fname = "/home/saniya/Code/pythonprojects/leaf/images/" + my_image
image = np.array(ndimage.imread(fname, flatten=False))
my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((1, num_px*num_px*3)).T
my_predicted_image = predict(d["w"], d["b"], my_image)

plt.imshow(image)
print("y = " + str(np.squeeze(my_predicted_image)) + ", your algorithm predicts a \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" picture.")

ValueError: cannot reshape array of size 4096 into shape (1,12288)

In [20]:
image = np.array(ndimage.imread(fname, flatten=False))

In [18]:
image.shape

(251, 667)

In [21]:
image.shape

(467, 526)

In [22]:
251 * 667

167417

In [23]:
467 * 526

245642

In [24]:
my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((1, num_px*num_px*3)).T

ValueError: cannot reshape array of size 4096 into shape (1,12288)

In [40]:
def resize_image(img, max_dim=96):
    """
    Resize the image so the maximum side of the size max_dim
    Returns a new image of the right size
    """
    max_ax = max((0,1), key=lambda i: img.size[i])
    scale = max_dim/float(img.size[max_ax])
    return img.resize((int(img.size[0] * scale), int(img.size[1] * scale)))

In [50]:
my_image = "5.jpg"  

# Preprocess the image to fit the algorithm.
fname = "/home/saniya/Code/pythonprojects/leaf/images/" + my_image


In [30]:
from keras.preprocessing.image import img_to_array, load_img

In [51]:
image = load_img(fname, grayscale=True)

In [52]:
x = resize_image(image, max_dim=96)
x = img_to_array(x)

In [44]:
x.shape

(63, 96, 1)

In [53]:
length = x.shape[0]
width = x.shape[1]

In [54]:
max_dim = 96
h1 = int((max_dim - length) / 2)
h2 = h1 + length
w1 = int((max_dim - width) / 2)
w2 = w1 + width

In [55]:
h1

4